<a href="https://colab.research.google.com/github/Zarif123/SSLM-Project/blob/main/chess_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%%capture
!pip install transformers
!pip install python-chess

In [10]:
from datetime import datetime

from google.colab import drive
drive.mount('/content/gdrive')
folder = "/content/gdrive/MyDrive/Statistical_LM_Group_Folder"

csv_file = "chess_data_with_buckets.csv"
csv_path = f"{folder}/{csv_file}"


now = datetime.now()
dt_string = now.strftime("%m_%d_%Y_%H_%M")

model_file = "classifier_model.pth"
datetime = dt_string
model_path = f"{folder}/{datetime}_{model_file}"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, get_linear_schedule_with_warmup
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from imblearn.under_sampling import RandomUnderSampler

###Parameters

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

params = {
    "batch_size": 2,
    "epochs": 5,
    "learning_rate": 0.005,
    "warmup_steps": 0.01,
    "epsilon": 1e-8,
    "accum_iter": 4
}

class Params:
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

params = Params(**params)

### Classify Model

In [13]:
class BertClassifier(nn.Module):
  def __init__(self, dropout=0.5, num_classes=11):
    super(BertClassifier, self).__init__()

    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(768, num_classes)
    self.relu = nn.ReLU()

  def forward(self, input_id, mask):
    _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
    dropout_output = self.dropout(pooled_output)
    linear_output = self.linear(dropout_output)
    final_layer = self.relu(linear_output)

    return final_layer

### Dataset Class

In [14]:
class ChessDataset(torch.utils.data.Dataset):
  def __init__(self, moves, labels):
      tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
      self.labels = torch.Tensor(labels.values)
      self.moves = [tokenizer(move, 
                              padding='max_length', max_length = 512, truncation=True,
                              return_tensors="pt") for move in moves]

  def __len__(self):
      return len(self.labels)

  def __getitem__(self, idx):
      return self.moves[idx], self.labels[idx]

### Training Function

In [22]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer):
    num_batches = len(train_dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = params.warmup_steps, num_training_steps = num_batches * params.epochs)
    for epoch_num in range(params.epochs):
        total_acc_train = 0
        total_loss_train = 0

        for batch_idx, (train_input, train_label) in enumerate(train_dataloader):
          if batch_idx % 10 == 0:
            print(f"Batch Number: {batch_idx}")
          train_label = train_label.to(device)
          mask = train_input['attention_mask'].to(device)
          input_id = train_input['input_ids'].squeeze(1).to(device)

          output = model(input_id, mask)
          
          batch_loss = criterion(output, train_label.long())
          total_loss_train += batch_loss.item()
          
          acc = (output.argmax(dim=1) == train_label).sum().item()
          total_acc_train += acc

          batch_loss.backward()

          # Gradient Accumulation
          if ((batch_idx + 1) % params.accum_iter == 0) or (batch_idx + 1 == num_batches):
            optimizer.step()
            scheduler.step()
            model.zero_grad()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

          for val_input, val_label in val_dataloader:
            val_label = val_label.to(device)
            mask = val_input['attention_mask'].to(device)
            input_id = val_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, val_label.long())
            total_loss_val += batch_loss.item()
            
            acc = (output.argmax(dim=1) == val_label).sum().item()
            total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataloader): .3f} \
            | Train Accuracy: {total_acc_train / len(train_dataloader): .3f} \
            | Val Loss: {total_loss_val / len(val_dataloader): .3f} \
            | Val Accuracy: {total_acc_val / len(val_dataloader): .3f}')
        
        torch.save(model.state_dict(), model_path)

### Test Function

In [23]:
def test(model, test_dataloader):
  total_acc_test = 0
  with torch.no_grad():
    for test_input, test_label in test_dataloader:
      test_label = test_label.to(device)
      mask = test_input['attention_mask'].to(device)
      input_id = test_input['input_ids'].squeeze(1).to(device)

      output = model(input_id, mask)
      print(output.argmax(dim=1), test_label)

      acc = (output.argmax(dim=1) == test_label).sum().item()
      total_acc_test += acc
      
      print(f"Test Accuracy: {total_acc_test / len(test_dataloader): .3f}")

### Loading Data

In [34]:
%%capture
chess_data = pd.read_csv(csv_path)[0:30000]
X = chess_data["Moves"]
y = chess_data["Bucket"]

balancer = RandomUnderSampler()
X = X.values.reshape(-1, 1)
X, y = balancer.fit_resample(X, y)
rating_counts = y.value_counts().sort_index()
X = X.flatten()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, shuffle=True) # Splits into train/test
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, shuffle=True) # Splits train into train/val

train_data, val_data, test_data = ChessDataset(X_train, y_train), ChessDataset(X_val, y_val), ChessDataset(X_test, y_test)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=params.batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=params.batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=params.batch_size)

In [35]:
%%capture
model = BertClassifier(num_classes=14)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = params.learning_rate)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Run Training

In [26]:
train(model, train_dataloader, val_dataloader, criterion, optimizer)

Batch Number: 0
Batch Number: 10
Batch Number: 20
Batch Number: 30
Batch Number: 40
Batch Number: 50
Batch Number: 60
Batch Number: 70
Batch Number: 80
Batch Number: 90
Batch Number: 100
Batch Number: 110
Batch Number: 120
Batch Number: 130
Batch Number: 140
Epochs: 1 | Train Loss:  3.318             | Train Accuracy:  1.340             | Val Loss:  3.182             | Val Accuracy:  1.200
Batch Number: 0
Batch Number: 10
Batch Number: 20
Batch Number: 30
Batch Number: 40
Batch Number: 50
Batch Number: 60
Batch Number: 70
Batch Number: 80
Batch Number: 90
Batch Number: 100
Batch Number: 110
Batch Number: 120
Batch Number: 130
Batch Number: 140
Epochs: 2 | Train Loss:  3.085             | Train Accuracy:  1.373             | Val Loss:  3.074             | Val Accuracy:  1.560
Batch Number: 0
Batch Number: 10
Batch Number: 20
Batch Number: 30
Batch Number: 40
Batch Number: 50
Batch Number: 60
Batch Number: 70
Batch Number: 80
Batch Number: 90
Batch Number: 100
Batch Number: 110
Batch Num

In [2]:
test_model = model.load_state_dict(torch.load(model_path))
test(test_model, test_dataloader)

NameError: ignored